<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [2]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [2]:
#!head -1 ../../data/train.json > ../../data/train1.json
!head -1 './habr-num-bookmarks/train.json' > train1.json

In [46]:
with open('train1.json') as inp_json:
    first_json = json.load(inp_json)

In [47]:
first_json.keys()

dict_keys(['author', '_timestamp', 'post_id', 'published', 'title', 'flow', 'polling', 'meta_tags', 'flags', 'domain', '_id', 'link_tags', 'content', 'tags', 'hubs', 'url'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [6]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [7]:
first_json['_timestamp']

1493192186.0903192

In [103]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [9]:
first_json['domain']

'habrahabr.ru'

In [10]:
first_json['published']

{'$date': '2006-07-15T01:48:00.000Z'}

In [3]:
from dateutil import parser
parser.parse(first_json['published']['$date']).month
parser.parse(first_json['published']['$date']).hour

NameError: name 'first_json' is not defined

In [11]:
first_json['title']

'Самопроизвольное разлогинивание'

In [12]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [13]:
first_json['polling']

In [14]:
first_json['post_id']

7

In [15]:
first_json['flags']

[]

In [16]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [73]:
first_json['hubs'][0]['title']

'Хабрахабр'

In [17]:
first_json['flow']

In [18]:
first_json['tags']

['логин', 'login']

In [48]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [66]:
first_json['author']['nickname'].replace("''",'')

'@ptitov'

In [20]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [21]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [4]:
#train_target = pd.read_csv('../../data/train_target.csv', index_col='url')
train_target = pd.read_csv('./train_target.csv', index_col='url')

In [39]:
train_target.head()

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000


In [5]:
train_target.loc['https://habrahabr.ru/post/7/']['target']

0.69314700000000007

Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину: постройте признак content_len – длина текста в миллионах символов.
Также постройте признаки: час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [13]:
import re

In [28]:
re.sub('[^a-zA-Z1-9а-яА-Я ]', '', 
       'title Подробности о Medison Celebrity: халявы не будет!?'.lower())

'title подробности о medison celebrity халявы не будет'

In [7]:
#для теста делаем файл с 1000ю записями
!head -1000 './habr-num-bookmarks/train.json' > train1000_zm.json

In [34]:
#вот этот код 1000!
import re
from dateutil import parser
i = 0
with open('./train1000_zm.json', encoding='utf-8') as inp_json, open('train_zm_1000.vw', 'w', encoding='utf-8') as out_vw, open('train_zm_valid_labels_1000.vw', 'w') as out_vw_labels:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)        
        if i<700:
            target = train_target.loc[data_json['url']]['target']
        else:
            target = 1
        out_vw.write('{} |title {} |tags {} |domain {} |flow {} |author {} |hubs {} |num content_len:{} month:{} hour:{}\n'
              .format(round(target, 6)
                , re.sub('[^a-zA-Z1-9а-яА-Я ]', '', data_json['title'].lower())
                , ' '.join(data_json['tags'])
                , data_json['domain'], data_json['flow']
                , data_json['author']['nickname']
                , data_json['hubs'][0]['title']
                , round(len(data_json['content']) / 1000000, 2)
                , parser.parse(data_json['published']['$date']).month
                , parser.parse(data_json['published']['$date']).hour
                )
             )
        out_vw_labels.write('{}\n'.format(round(train_target.loc[data_json['url']]['target'], 6)))
        i+=1;

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [35]:
!wc -l *1000*.vw

  1000 train_zm_1000.vw
  1000 train_zm_valid_labels_1000.vw
  2000 total


In [36]:
!split -l 700 './train_zm_1000.vw' train_zm_1000_

In [37]:
!split -l 700 './train_zm_valid_labels_1000.vw' train_zm_valid_labels_1000_

------------

In [12]:
import re
from dateutil import parser
i = 0
with open('./habr-num-bookmarks/train.json', encoding='utf-8') as inp_json, open('train_zm_main.vw', 'w', encoding='utf-8') as out_vw, open('train_zm_valid_labels_main.vw', 'w') as out_vw_labels:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)        
        if i<84000:
            target = train_target.loc[data_json['url']]['target']
        else:
            target = 1
        out_vw.write('{} |title {} |tags {} |domain {} |flow {} |author {} |hubs {} |num content_len:{} month:{} hour:{}\n'
              .format(round(target, 6)
                #, re.sub('[^a-zA-Z1-9а-яА-Я +@#]', '', data_json['title'].lower())
                , re.sub('[^a-zA-Z1-9а-яА-Я +@#]', '', data_json['title'].lower())                      
                , re.sub('[^a-zA-Z1-9а-яА-Я +@#]', '', ' '.join(data_json['tags']).lower())
                , data_json['domain'], data_json['flow']
                , data_json['author']['nickname']
                , re.sub('[^a-zA-Z1-9а-яА-Я +@#]', '', data_json['hubs'][0]['title'].lower())
                , round(len(data_json['content']) / 1000000, 3)
                , parser.parse(data_json['published']['$date']).month
                , parser.parse(data_json['published']['$date']).hour
                )
             )
        out_vw_labels.write('{}\n'.format(round(train_target.loc[data_json['url']]['target'], 6)))
        i+=1;

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [13]:
!split -l 84000 './train_zm_main.vw' train_zm_main_

In [14]:
!split -l 84000 './train_zm_valid_labels_main.vw' train_zm_valid_labels_main_

In [15]:
!wc -l *zm_*main*

    52913 test_zm_main.vw
   120000 train_zm_main.vw
    84000 train_zm_main_aa
    36000 train_zm_main_ab
    36000 train_zm_main_test.vw
    84000 train_zm_main_train.vw
   120000 train_zm_valid_labels_main.vw
    84000 train_zm_valid_labels_main_aa
    36000 train_zm_valid_labels_main_ab
    36000 train_zm_valid_labels_main_test.vw
    84000 train_zm_valid_labels_main_train.vw
   772913 total


In [16]:
!mv train_zm_main_aa train_zm_main_train.vw
!mv train_zm_main_ab train_zm_main_test.vw
!mv train_zm_valid_labels_main_aa train_zm_valid_labels_main_train.vw
!mv train_zm_valid_labels_main_ab train_zm_valid_labels_main_test.vw

In [17]:
!wc -l *zm_*main*

   52913 test_zm_main.vw
  120000 train_zm_main.vw
   36000 train_zm_main_test.vw
   84000 train_zm_main_train.vw
  120000 train_zm_valid_labels_main.vw
   36000 train_zm_valid_labels_main_test.vw
   84000 train_zm_valid_labels_main_train.vw
  532913 total


In [11]:
#тестовая
import re
from dateutil import parser
i = 0
target = 1
with open('./habr-num-bookmarks/test.json', encoding='utf-8') as inp_json, open('test_zm_main.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)        
        out_vw.write('{} |title {} |tags {} |domain {} |flow {} |author {} |hubs {} |num content_len:{} month:{} hour:{}\n'
              .format(round(target, 6)
                , re.sub('[^a-zA-Z1-9а-яА-Я +@#]', '', data_json['title'].lower())
                , re.sub('[^a-zA-Z1-9а-яА-Я +@#]', '', ' '.join(data_json['tags']).lower())
                , data_json['domain'], data_json['flow']
                , data_json['author']['nickname']
                , re.sub('[^a-zA-Z1-9а-яА-Я +@#]', '', data_json['hubs'][0]['title'].lower())
                , round(len(data_json['content']) / 1000000, 3)
                , parser.parse(data_json['published']['$date']).month
                , parser.parse(data_json['published']['$date']).hour
                )
             )

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

----

In [26]:
#для теста делаем файл с 2мя записями
!head -2 './habr-num-bookmarks/train.json' > train2_zm.json

In [4]:
with open('train2_zm.vw', 'w') as out_file:
    out_file.write('{} | {}\n'.format(str(1), str('Hello')))
               
#               format(str(topic_map[list(topics_from_list)[0]]), 
#                                              text.strip().replace(':', '').replace('|', '')))

In [126]:
from dateutil import parser
with open('train2_zm.json') as inp_json, open('train2_zm.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        print('{} |title {} |tags {} |domain {} |flow {} |author {} |hubs {} |num content_len:{} month:{} hour:{}\n'
              .format(train_target.loc[data_json['url']]['target']
                , data_json['title']
                , ' '.join(data_json['tags'])
                , data_json['domain'], data_json['flow']
                , data_json['author']['nickname']
                , data_json['hubs'][0]['title']
                , round(len(data_json['content']) / 1000000, 1)
                , parser.parse(data_json['published']['$date']).month
                , parser.parse(data_json['published']['$date']).hour
                )
             )
        #out_vw.write('{}|{}\n'.format(data_json['title'], str(data_json['tags']).replace('''''', '')))
        

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

0.6931470000000001 |title Самопроизвольное разлогинивание |tags логин login |domain habrahabr.ru |flow None |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1

1.0986120000000001 |title Stand-along cообщества против сообществ в рамках социальных сетей |tags сообщества интернет-сообщество социальные сети нишевой бренд |domain geektimes.ru |flow None |author @AlexBruce |hubs Чёрная дыра |num content_len:0.0 month:7 hour:14




In [127]:
from dateutil import parser
with open('train2_zm.json') as inp_json, open('train2_zm.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        out_vw.write('{} |title {} |tags {} |domain {} |flow {} |author {} |hubs {} |num content_len:{} month:{} hour:{}\n'
              .format(train_target.loc[data_json['url']]['target']
                , data_json['title']
                , ' '.join(data_json['tags'])
                , data_json['domain'], data_json['flow']
                , data_json['author']['nickname']
                , data_json['hubs'][0]['title']
                , round(len(data_json['content']) / 1000000, 1)
                , parser.parse(data_json['published']['$date']).month
                , parser.parse(data_json['published']['$date']).hour
                )
             )

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [11]:
#вот этот код!
from dateutil import parser
i = 0
with open('./train1000_zm.json.json', encoding='utf-8') as inp_json, open('train_zm_1000.vw', 'w', encoding='utf-8') as out_vw, open('train_zm_valid_labels.vw', 'w') as out_vw_labels:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        i+=1;
        if i<84000:
            target = train_target.loc[data_json['url']]['target']
        else:
            target = 1
        out_vw.write('{} |title {} |tags {} |domain {} |flow {} |author {} |hubs {} |num content_len:{} month:{} hour:{}\n'
              .format(round(target, 6)
                , data_json['title']
                , ' '.join(data_json['tags'])
                , data_json['domain'], data_json['flow']
                , data_json['author']['nickname']
                , data_json['hubs'][0]['title']
                , round(len(data_json['content']) / 1000000, 1)
                , parser.parse(data_json['published']['$date']).month
                , parser.parse(data_json['published']['$date']).hour
                )
             )
        out_vw_labels.write('{}\n'.format(train_target.loc[data_json['url']]['target']))

In [129]:

#тестовая
with open('./habr-num-bookmarks/test.json') as inp_json, open('test_zm.vw', 'w', encoding='utf-8') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        out_vw.write('{} |title {} |tags {} |domain {} |flow {} |author {} |hubs {} |num content_len:{} month:{} hour:{}\n'
              .format('1'
                , data_json['title']
                , ' '.join(data_json['tags'])
                , data_json['domain'], data_json['flow']
                , data_json['author']['nickname']
                , data_json['hubs'][0]['title']
                , round(len(data_json['content']) / 1000000, 1)
                , parser.parse(data_json['published']['$date']).month
                , parser.parse(data_json['published']['$date']).hour
                )
             )

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [24]:
!head -2 ../../data/habr_train.vw

head: cannot open '../../data/habr_train.vw' for reading: No such file or directory


In [ ]:
with open('../../data/train.json') as inp_json, \
     open('../../data/habr_train.vw', 'w') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        
        # Ваш код здесь

Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

In [ ]:
with open('../../data/test.json') as inp_json, \
     open('../../data/habr_test.vw', 'w') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        
        # Ваш код здесь

In [24]:
!head -2 ../../data/habr_test.vw

1 |title День Пи! |tags Пи Pi |domain geektimes.ru |flow None |author @Timursan |hubs Чёрная дыра |num content_len:0.0 month:3 hour:3
1 |title Скрипт для разбиения образов музыкальных CD на треки и конвертации в формат FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs Чёрная дыра |num content_len:0.01 month:3 hour:0


In [130]:
!head -2 './test_zm.vw'

1 |title �������� ����! |tags ���� Pi |domain geektimes.ru |flow None |author @Timursan |hubs ������������ �������� |num content_len:0.0 month:3 hour:3
1 |title ������������ ������ ������������������ �������������� ���������������������� CD ���� ���������� �� ���������������������� �� ������������ FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs ������������ �������� |num content_len:0.0 month:3 hour:0


Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

In [138]:
!wc -l *zm.vw

   52913 test_zm.vw
      10 train10_zm.vw
       2 train2_zm.vw
  120000 train_zm.vw
  172925 total


In [135]:
120000 - 120000 * 0.3

84000.0

In [137]:
#делим выборку на обучающую и проверочную
!split -l 84000 './train_zm.vw' train_zm_

In [139]:
!wc -l *train_zm_*

   84000 train_zm_aa
   36000 train_zm_ab
  120000 total


In [140]:
!mv train_zm_aa train_zm_train
!mv train_zm_ab train_zm_test

In [ ]:
# Ваш код здесь

In [25]:
sample_sub = pd.read_csv('../../data/sample_submission.csv', 
                         index_col='url')

In [26]:
sample_sub.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [ ]:
your_submission = sample_sub.copy()
your_submission['target'] = # Ваш код здесь
your_submission.to_csv('submission.csv')

In [13]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import mean_absolute_error
import itertools

def train_vw_model(train_vw_file, model_filename, 
                   ngram=1, bit_precision=28, passes=1,
                   seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('vw {train_vw_file} ' + 
                       '-f {model_filename} -b {bit_precision} --random_seed {seed}').format(
                       train_vw_file=train_vw_file, 
                       model_filename=model_filename, bit_precision=bit_precision, seed=seed)
    if ngram > 1:
         vw_call_string += ' --ngram={}'.format(ngram)
            
    if passes > 1:
         vw_call_string += ' -k --passes={} --cache_file {}'.format(passes, 
                            model_filename.replace('.vw', '.cache'))
    if quiet:
        vw_call_string += ' --quiet'
    
    
    print(vw_call_string) 
    res = os.system(vw_call_string)
    print('Success. Elapsed: {} sec.'.format(round(time() - init_time, 2))
          if not res else 'Failed.')

def test_vw_model(model_filename, test_vw_file, prediction_filename,
                  true_labels, seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('vw -t -i {model_filename} {test_vw_file} ' + 
                       '-p {prediction_filename} --random_seed {seed}').format(
                       model_filename=model_filename, test_vw_file=test_vw_file, 
                       prediction_filename=prediction_filename, seed=seed)
    if quiet:
        vw_call_string += ' --quiet'
        
    print(vw_call_string) 
    res = os.system(vw_call_string)
    
    if not res: # the call resulted OK
        vw_pred = np.loadtxt(prediction_filename)
        print("Accuracy: {}%. Elapsed: {} sec.".format(
            round(100 * mean_absolute_error(true_labels, vw_pred), 2), 
            round(time() - init_time, 2)))
    else:
        print('Failed.')

y_valid = np.loadtxt('train_zm_valid_labels_test.vw')

In [14]:
y_valid

array([ 2.772589,  1.386294,  0.693147, ...,  3.044522,  4.110874,
        1.386294])

In [15]:
for i, (ngram, passes) in tqdm(enumerate(itertools.product([1,2,3],[1,3,5]))):
    train_vw_model('train_zm_train.vw', 
                   'vw_model{}_part.vw'.format(i), 
                   ngram=ngram, passes=passes,
                   bit_precision=28, 
                   seed=17, quiet=True)
    test_vw_model(model_filename='vw_model{}_part.vw'.format(i), 
              test_vw_file='train_zm_test.vw', 
              prediction_filename='vw_valid_pred{}.csv'.format(i),
              true_labels=y_valid, seed=17, quiet=True)

0it [00:00, ?it/s]

vw train_zm_train.vw -f vw_model0_part.vw -b 28 --random_seed 17 --quiet
Success. Elapsed: 184.45 sec.
vw -t -i vw_model0_part.vw train_zm_test.vw -p vw_valid_pred0.csv --random_seed 17 --quiet


FileNotFoundError: [Errno 2] No such file or directory: 'vw_valid_pred0.csv'

In [ ]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import mean_absolute_error
import itertools

best_mae = 5
best_str = ''
prev_str = ''

def train_vw_model(train_vw_file, model_filename,
                   ngram=1, bit_precision=27, passes=1,
                   learning_rate=0.5,
                   power_t = 0.5,
                   seed=17, quiet=True):
    init_time = time()
    global prev_str
    vw_call_string = ('vw {train_vw_file} ' + 
                       '-f {model_filename} -b {bit_precision} --random_seed {seed}' +
                      ' -l {learning_rate} --power_t {power_t} --loss_function quantile ' #--loss_function quantile
                     ).format(
                       train_vw_file=train_vw_file, learning_rate=learning_rate, 
                       model_filename=model_filename, bit_precision=bit_precision, seed=seed, power_t=power_t)
    if ngram > 1:
         vw_call_string += ' --ngram={}'.format(ngram)
            
    if passes > 1:
         vw_call_string += ' -k --passes={} --cache_file {}'.format(passes, 
                            model_filename.replace('.vw', '.cache'))
    if quiet:
        vw_call_string += ' --quiet'
    #`bits`, `learning_rate` и `power_t`
    
    print(vw_call_string) 
    prev_str = vw_call_string
    res = os.system(vw_call_string)
    print('Success. Elapsed: {} sec.'.format(round(time() - init_time, 2))
          if not res else 'Failed.')

def test_vw_model(model_filename, test_vw_file, prediction_filename,
                  true_labels, seed=17, quiet=True):
    global best_mae
    global best_str
    init_time = time()
    vw_call_string = ('vw -t -i {model_filename} {test_vw_file} ' + 
                       '-p {prediction_filename} --random_seed {seed}').format(
                       model_filename=model_filename, test_vw_file=test_vw_file, 
                       prediction_filename=prediction_filename, seed=seed)
    if quiet:
        vw_call_string += ' --quiet'
        
    print(vw_call_string) 
    res = os.system(vw_call_string)
    
    if not res: # the call resulted OK
        vw_pred = np.loadtxt(prediction_filename)
        print("MAE: {}. Elapsed: {} sec.".format(
            round(mean_absolute_error(true_labels, vw_pred), 6), 
            round(time() - init_time, 2)))
    else:
        print('Failed.')
    if best_mae > round(mean_absolute_error(true_labels, vw_pred), 6):
        best_mae = round(mean_absolute_error(true_labels, vw_pred), 6)
        best_str = prev_str
        print('MAE = ', best_mae)
        print('str:\n', best_str)
y_valid = np.loadtxt('train_zm_valid_labels_main.vw')

for i, (ngram, passes, bit_precision, learning_rate, power_t) in tqdm(enumerate(itertools.product([2,3],[3],[27],[0.1817],[0.855]))):
    train_vw_model('train_zm_main.vw', 
                   'vw_model{}_part.vw'.format(i), 
                   ngram=ngram, passes=passes,
                   bit_precision = bit_precision,
                   learning_rate = learning_rate, power_t = power_t, 
                   seed=17, quiet=True)
    test_vw_model(model_filename='vw_model{}_part.vw'.format(i), 
              test_vw_file='test_zm_main.vw', 
              prediction_filename='vw_valid_pred{}.csv'.format(i),
              true_labels=y_valid, seed=17, quiet=True)

print('MAE = ', best_mae)
print('str:\n', best_str)

0it [00:00, ?it/s]

vw train_zm_main.vw -f vw_model0_part.vw -b 27 --random_seed 17 -l 0.1817 --power_t 0.855 --loss_function quantile  --ngram=2 -k --passes=3 --cache_file vw_model0_part.cache --quiet
Success. Elapsed: 7.12 sec.
vw -t -i vw_model0_part.vw test_zm_main.vw -p vw_valid_pred0.csv --random_seed 17 --quiet


1it [00:07,  7.91s/it]

MAE: 0.788385. Elapsed: 0.79 sec.
MAE =  0.788385
str:
 vw train_zm_main.vw -f vw_model0_part.vw -b 27 --random_seed 17 -l 0.1817 --power_t 0.855 --loss_function quantile  --ngram=2 -k --passes=3 --cache_file vw_model0_part.cache --quiet
vw train_zm_main.vw -f vw_model1_part.vw -b 27 --random_seed 17 -l 0.1817 --power_t 0.855 --loss_function quantile  --ngram=3 -k --passes=3 --cache_file vw_model1_part.cache --quiet
Failed.
vw -t -i vw_model1_part.vw test_zm_main.vw -p vw_valid_pred1.csv --random_seed 17 --quiet


In [22]:
!vw train_zm_all.vw -f vw_model_part.vw -b 26 --random_seed 17 -l 1 --power_t 0.855  --ngram=2 -k --passes=3 --cache_file vw_model_part.cache --quiet

In [23]:
os.system('vw train_zm_all.vw -f vw_model_part.vw -b 26 --random_seed 17 -l 1 --power_t 0.855  --ngram=2 -k --passes=3 --cache_file vw_model_part.cache --quiet')

0

In [24]:
os.system('vw -t -i vw_model_part.vw test_zm_main_final.vw -p vw_valid_pred.csv --random_seed 17 --quiet')

0

Для получения баллов в #mlcourse_open команда (из 1 человека) должна называться в точном соответствии с тем, как оно записано в рейтинге.